In [1]:
#!pip install folium
#!pip install plotly

In [2]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import plotly.graph_objects as go


In [3]:
data = pd.read_csv('ModeloBayesianoMarzoMayo2020_2.csv')

# cast errores de captura
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'CIUDAD DE MEXICO'] = 'CIUDAD DE MÉXICO'
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'TUXTLA GUTIÉRREZ'] = 'TUXTLA GUTIERREZ'
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'CIUDAD VICTORIA'] = 'TAMAULIPAS'
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'SAN LUIS POTOSI'] = 'SAN LUIS POTOSÍ'
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'CIUDAD VICTORIA'] = 'TAMAULIPAS'
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'TIJUANA'] = 'BAJA CALIFORNIA'
data['DESC_CE_FINANCIERO'][data['DESC_CE_FINANCIERO'] == 'QUERÉTARO'] = 'QUERETARO'

C:\Users\fou-f\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\fou-f\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\fou-f\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\fou-f\anaconda3\lib\site-packages\ipykernel_l

In [4]:
mapeo = pd.read_csv('JoinCoordenadas.csv')
data = data.merge(mapeo, how='right', on='DESC_CE_FINANCIERO' )
data = data.dropna()
data['FECHA_TRANSACCION'] = pd.to_datetime(data.FECHA_TRANSACCION, format='%d/%m/%Y')

In [5]:
#data.to_csv('input_plots.csv')
data['LONG'] = -data.LONG
datos = data[['CLAVE_CLIENTE', 'FECHA_TRANSACCION', 'ENTIDAD_FEDERATIVA', 'LAT', 'LONG']]

In [6]:
mapa = datos.groupby(['ENTIDAD_FEDERATIVA', 'LAT', 'LONG']).size().reset_index()
mapa.columns = ['ENTIDAD_FEDERATIVA', 'LAT', 'LONG', 'Conteo']
mapa = mapa.sort_values(['Conteo'],ascending=False).reset_index()

In [7]:
mapit = folium.Map( location = [25.,-102.], zoom_start = 6)
folium.TileLayer('cartodbpositron').add_to(mapit)

for idx in range(0,len(mapa) ):
    # como no todos los estados se ven bien los marco nada mas 
    folium.Circle(     radius=100, location=mapa[  ['LAT', 'LONG'] ].iloc[idx],
                       popup=str(mapa['ENTIDAD_FEDERATIVA'][ idx ]),  color='#0b3c5d',
                       fill=False, ).add_to(mapit)
    
    folium.CircleMarker(  location=mapa[  ['LAT', 'LONG'] ].iloc[idx],
                            radius=np.log(int(mapa[ ['Conteo']].iloc[idx]))*7  , 
                            color= '#0b3c5d',#'#d9b310',
                            fill=True, fill_color='#329cc1',
                            popup=str(mapa['ENTIDAD_FEDERATIVA'][ idx ])).add_to(mapit)
    
mapit

In [8]:
tiempo = datos.groupby(['FECHA_TRANSACCION'] ).size().reset_index()
tiempo.columns = ['FECHA_TRANSACCION', 'OperacionesDiarias']
tiempo = tiempo.sort_values(['FECHA_TRANSACCION'],ascending=True).reset_index()

In [9]:
layout = go.Layout(
    plot_bgcolor='rgba(0,0,0,0)', title = 'Operaciones electrónicas por día (feb-may, 2020)'
)
fig = go.Figure(data=go.Scatter(x=tiempo['FECHA_TRANSACCION'], y=tiempo['OperacionesDiarias'],
                                mode='lines', line_color='#328cc1'), layout=layout)
fig.show()